In [1]:
import pandas as pd
from fuzzywuzzy import fuzz

def merge(entry, column, fuzz_ratio, remove = [], include = []):
    variants = []
    for x in job_posts[column].unique():
        if fuzz.ratio(x, entry) > fuzz_ratio:
            variants.append(x)
    for element in remove:
        variants.remove(element)
    for element in include:
        variants.append(element)

    return job_posts[column].map(lambda s: entry if s in variants else s)

# job_posts = pd.read_csv('/kaggle/input/job-posts/job-posts.csv')
job_posts = pd.read_csv('/kaggle/input/scrape-cscjobportal/job-posts.csv')
# job_posts = pd.read_csv('/kaggle/input/csc-job-portal-scraping/job-posts.csv')

# drop unwanted cols
# job_posts = job_posts[[
#     'place_of_assignment',
#     'position_title',
#     'plantilla_item_no',
#     'salary_grade',
#     'monthly_salary'
# ]]

# drop empty posts
job_posts = job_posts.loc[(job_posts['position_title'].notna()) & (job_posts.closing_date.notna())]

job_posts.education = job_posts.education.fillna('-')
job_posts.training = job_posts.training.fillna('-')
job_posts.work_experience = job_posts.work_experience.fillna('-')
job_posts.eligibility = job_posts.eligibility.fillna('-')

# drop posts with invalid dates
job_posts = job_posts.loc[job_posts.closing_date != 'November 30, -0001']
job_posts = job_posts.loc[job_posts.posting_date != 'November 30, -0001']

# fix the index
job_posts = job_posts.loc[job_posts['Unnamed: 0'].notna()]
job_posts = job_posts.astype({'Unnamed: 0': 'int64'})
job_posts = job_posts.rename(columns={'Unnamed: 0': 'job_post_id'})
job_posts.set_index('job_post_id', inplace=True)

# clean columns
job_posts = job_posts.loc[job_posts['http_status_code'].notna()]
job_posts = job_posts.astype({'http_status_code': 'int64'})

job_posts['monthly_salary'] = job_posts['monthly_salary'].str.replace('Php ', '').str.replace(',', '').astype(float)

job_posts.closing_date = pd.to_datetime(job_posts.closing_date)
job_posts.posting_date = pd.to_datetime(job_posts.posting_date)

job_posts.education = merge("Bachelor's Degree", "education", 80)
job_posts.education = merge('Doctor of Medicine', 'education', 90)
job_posts.education = merge('Completion of two years studies in college', 'education', 90)
job_posts.education = merge("Bachelor's degree relevant to the job", 'education', 89)
job_posts.education = merge("Elementary School Graduate", 'education', 87)
job_posts.education = merge("Bachelor of Science in Nursing", 'education', 78)
job_posts.education = merge("Must be able to read and write", 'education', 71)
job_posts.education = merge("High School Graduate", 'education', 74)

job_posts.training = merge("None required", 'training', 52, include = ['NONE REQUIRED'])
job_posts.training = merge("4 hours of relevant training", 'training', 97, ['40 hours of relevant training', '24 hours of relevant training', '45 hours of relevant training', '48 hours of relevant training'], ['Four (4) hours of relevant training', '4 HOURS OF RELEVANT TRAINING', '4 hours relevant training'])
job_posts.training = merge("8 hours of relevant training", 'training', 97, ['80 hours of relevant training', '28 hours of relevant training', '18 hours of relevant training', '48 hours of relevant training'])

job_posts.work_experience = merge("None required", 'work_experience', 77, include = ['NONE REQUIRED'])
job_posts.work_experience = merge("1 year of relevant experience", 'work_experience', 96, remove = ['4 year of relevant experience', '3 year of relevant experience', '2 year of relevant experience', '16 year of relevant experience'], include = ['1 year relevant experience', '1 YEAR OF RELEVANT EXPERIENCE', 'One (1) year of relevant experience'])
job_posts.work_experience = merge("2 years of relevant experience", 'work_experience', 97, include = ['2 years relevant experience', '2 YEARS OF RELEVANT EXPERIENCE', 'Two (2) years of relevant experience'])
job_posts.work_experience = merge("3 years of relevant experience", 'work_experience', 97, include = ['3 years relevant experience'])

job_posts.eligibility = merge("RA 1080 (Teacher)", 'eligibility', 87, include = ['RA 1080 (TEACHER)'])
job_posts.eligibility = merge("Career Service (Professional) Second Level Eligibility", 'eligibility', 95, include = ['CSP/2ND LEVEL ELIGIBILITY', 'Career Service (Professional)/2nd Level Eligibility'])
job_posts.eligibility = merge("Career Service (Subprofessional) First Level Eligibility", 'eligibility', 95, include = ['CS-SP (1st Level Eligibility)', 'Career Service Subprofessional/\nFirst Level Eligibility'])
job_posts.eligibility = merge("None required", 'eligibility', 85)

# job_posts.position_title = merge("TEACHER I", 'position_title', 90, ['TEACHER II'], ['Teacher I'])
# job_posts.position_title = merge("TEACHER II", 'position_title', 91, ['TEACHER III', 'TEACHER I'], ['Teacher II'])
# job_posts.position_title = merge("TEACHER III", 'position_title', 91, ['TEACHER II'], ['Teacher III'])
# job_posts.position_title = merge("MEDICAL OFFICER III", 'position_title', 95, ['MEDICAL OFFICER II'], ['Medical Officer III'])
# job_posts.position_title = merge("MEDICAL OFFICER IV", 'position_title', 97, include = ['Medical Officer IV'])
# job_posts.position_title = merge("NURSE I", 'position_title', 97, include = ['Nurse I'])
# job_posts.position_title = merge("NURSE II", 'position_title', 97, include = ['Nurse II'])
# job_posts.position_title = merge("ADMINISTRATIVE ASSISTANT III", 'position_title', 99, include = ['Administrative Assistant III'])

job_posts[['position_title', 'monthly_salary', 'closing_date', 'plantilla_item_no', 'place_of_assignment', 'agency', 'region']].to_csv('cleaned-job-posts.csv')

In [ ]:
# job_posts[job_posts.position_title.str.contains('Computer Maintenance Technologist I')][['monthly_salary', 'education', 'training', 'work_experience']]

In [ ]:
job_posts.loc[(job_posts.closing_date == '2024-03-04') & (job_posts.region == 'Region I')][['position_title', 'monthly_salary', 'closing_date', 'plantilla_item_no', 'place_of_assignment', 'agency', 'region']].sort_values(by='monthly_salary').head()

In [6]:
len(job_posts.index.unique())

121716